<a href="https://colab.research.google.com/github/Anvayt24/X-RAY-Analysis-GAN-/blob/main/XRanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow


In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install matplotlib


In [ ]:
!pip install uvicorn


In [ ]:
!pip install fastapi

firstly step we have to load the data set and preprocess it for use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/ColabNotebooks/dataset.zip' -d '/content/dataset'

Archive:  /content/drive/MyDrive/ColabNotebooks/dataset.zip
replace /content/dataset/train/XR_ELBOW/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import cv2
import numpy as np


In [ ]:
img_size = 64
data_dir = '/content/dataset'

def load_data(data_dir , class_mapping):
  images=[]
  labels=[]

  for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
      label= class_mapping.get(folder, None)
      if label is not None:
        for image_file in os.listdir(folder_path):
          img_path = os.path.join(folder_path, image_file)
          img= cv2.imread(img_path , cv2.IMREAD_GRAYSCALE)
          img= cv2.resize(img ,(img_size , img_size))
          images.append(img)
          labels.append(label)
  images= np.array(images).astype("float32")/ 255.0
  labels = np.array(labels)
  return images , labels

class_mapping = {
    "XR_ELBOW" : 0,
    "XR_FINGER" : 1,
    "XR_FOREARM" : 2,
    "XR_HAND" : 3,
    "XR_HUMERUS" : 4,
    "XR_SHOULDER" : 5,
    "XR_WRIST" : 6
}

train_path = "/content/dataset/train"
test_path = "/content/dataset/valid"

X_train , y_train = load_data(train_path , class_mapping )
X_test , y_test = load_data(test_path , class_mapping )

X_train = np.expand_dims(X_train , axis=-1)
X_test = np.expand_dims(X_test , axis=-1)

print(f"Training data: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation data: {X_test.shape}, Labels: {y_test.shape}")



Training data: (15407, 64, 64, 1), Labels: (15407,)
Validation data: (1750, 64, 64, 1), Labels: (1750,)


training the CNN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

num_classes = len(class_mapping)
y_train = to_categorical(y_train , num_classes)
y_test = to_categorical(y_test , num_classes)

def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128,(3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax'),
    ])
    model.compile(optimizer= "adam" , loss ="categorical_crossentropy" , metrices =["accuracy"])
    return model

    cnn_model= create_cnn_model(input_shape=(img_size , img_size , 1))
    cnn_model.summary()

    history = cnn_model.fit(X_train , y_train , epochs=10 , validation_data=(X_test , y_test))
    cnn_model.save("cnn_model.h5")



creating synthetic data

In [ ]:
from tensorflow.keras.layers import Input , Dense, Reshape , Flatten , LeakyReLU , Dropout
from tensorflow.keras.models import Model
import tensorflow as tf

def generator_model(latent_dim):
  generator = Sequential([
      Dense(64, activation ="relu" , input_dim= latent_dim),
      LeakyReLU(alpha = 0.2),
      Dense(img_size * img_size  , activation ="tanh"),
      Reshape((img_size , img_size , 1))
  ])
  return generator

def discriminator_model(img_shape):
  discriminator = Sequential([
      Flatten(input_shape = img_shape),
      Dense(64 , activation ="relu"),
      LeakyReLU(alpha= 0.2),
      Dropout(0.3),
      Dense(1, activation ="sigmoid")
  ])
  discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
  discriminator.compile(loss="binary_crossentropy" ,  metrics=["accuracy"])
  return discriminator

combining generator and discriminator

In [ ]:
latent_dim = 100
generator = generator_model(latent_dim)
discriminator = discriminator_model(img_shape=(img_size , img_size , 1))
generator.summary()
discriminator.summary()

discriminator.trainable =False    # freezing the discrin=minator weights
gan_input = Input(shape = (latent_dim,))
fake_image = generator(gan_input)
validity = discriminator(fake_image)
gan = Model(gan_input , validity)
gan.compile(loss="binary_crossentropy" , optimizer="adam")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 4096)                │         266,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 64, 64, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 272,704 (1.04 MB)

 Trainable params: 272,704 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │         262,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 262,273 (1.00 MB)

 Trainable params: 262,273 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 1000
batch_size=16
real = np.ones((batch_size , 1))
fake = np.zeros((batch_size , 1))

for epochs in range(epochs):
  idx = np.random.randint(0,X_train.shape[0] , batch_size)
  real_images = X_train[idx]

  noise = np.random.normal(0,1,(batch_size , latent_dim))
  fake_images = generator.predict(noise)

  d_loss_real = discriminator.train_on_batch(real_images , real)
  d_loss_fake = discriminator.train_on_batch(fake_images , fake)
  d_loss = 0.5 * np.add(d_loss_real , d_loss_fake)

  noise = np.random.normal(0,1,(batch_size , latent_dim))
  g_loss = gan.train_on_batch(noise , real)

  if epochs %100 == 0 :
    print(f"{epochs} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6141700744628906, acc.: 70.3125] [G loss: [array(0.65375966, dtype=float32), array(0.65375966, dtype=float32), array(0.53125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

In [ ]:
print("Shape of X_test[0]:", X_test[0].shape)


Shape of X_test[0]: (64, 64, 1)


In [ ]:
OUTPUT_DIR = "generated_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def save_images(epoch):
    noise = np.random.normal(0, 1, (16, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]
    for i, img in enumerate(generated_images):
        plt.imsave(f"{OUTPUT_DIR}/generated_{epoch}_{i}.png", img.squeeze(), cmap="gray")


fastapi integration

In [ ]:
!pip install ngrok
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.0 MB/s eta 0:00:00


In [ ]:
!pip install nest_asyncio

In [ ]:
from fastapi import FastAPI, responses
from pydantic import BaseModel
from fastapi.responses import FileResponse
from pyngrok import ngrok
import uvicorn
from tensorflow.keras.preprocessing.image import array_to_img
import nest_asyncio

In [ ]:
app = FastAPI()

class GenerationRequest(BaseModel):
    num_images: int
ngrok.set_auth_token("2mvLFiwDjRpAvMmxyfFocPfLHD6_5Bw2mj2uBiQHMAXRfkqq7")


@app.post("/generate/")
def generate_images(request: GenerationRequest):
    num_images = request.num_images
    noise = np.random.normal(0, 1, (num_images,latent_dim))
    generated_images = generator.predict(noise)
    file_paths = []
    for i, img_array in enumerate(generated_images):
        img = array_to_img((img_array + 1) * 127.5)
        file_path = os.path.join(OUTPUT_DIR, f"generated_image_{i}.png")
        img.save(file_path)
        file_paths.append(file_path)
    return {"message": f"{num_images} images generated.", "file_paths": file_paths}

@app.get("/download/{image_name}")
def download_image(image_name: str):
    file_path = os.path.join(OUTPUT_DIR, image_name)
    if os.path.exists(file_path):
        return FileResponse(file_path, media_type="image/png", filename=image_name)
    return {"error": "File not found."}

@app.get("/")
async def root():
    return responses.RedirectResponse(url="/docs")

# Start the FastAPI server with ngrok
nest_asyncio.apply()
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [455]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://70bd-34-68-144-220.ngrok-free.app
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "POST /generate/ HTTP/1.1" 500 Internal Server Error


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, recei

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "POST /generate/ HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/fracture_123.png HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/fracture.png HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/0 HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/gen.PNG HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/0.png HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/generated_image_001.png HTTP/1.1" 200 OK
INFO:     2401:4900:5d86:672d:94e5:650f:aa84:8a03:0 - "GET /download/generated_image_1.png HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [455]
